# Computational Astrophysics 23-24
## Project track: Collisionless N-body simulations 
### Project-3: Tidal disruptions of a Plummer model 

## I Miloncini 

| Last Name | First Name | Student Number |
|-----------|------------|----------------|
|Bertinelli |Gabriele    |2103359         |
|Bonato     |Diego       |                |
|Di Prima   |Giacomo     |                |
|Viterbo    |Giuseppe    |                |

### Import modules

In [ ]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt, animation as animation, ticker as mticker
from matplotlib.pyplot import cm
import pandas as pd
from tqdm.notebook import tqdm

from fireworks.particles import Particles
import fireworks.nbodylib.nunits as NU
import fireworks.nbodylib.potentials as fnp
import fireworks.nbodylib.dynamics as fnd
import fireworks.nbodylib.integrators as fni
import fireworks.ic as ic

### Simulation

In [ ]:
# N = 500                # Number of stars in the cluster
# initial_position = 10   # Initial position along x direction 
# M_G = 1e3               # Mass of the galaxy
# eccentricity = 0.0    # Eccentricity of the orbit
# tstep = 0.01        # Time step for integration
# N_orbit = 2         # Number of orbits to integrate

# # Load data from CSV file
# path = './data/Plummer_Nbody_virial_' + str(N) + '.csv'
# data = pd.read_csv(path).values

# # Create Particles object
# particles = Particles(position=data[:, 4:], velocity=data[:, 1:4], mass=data[:, 0])

# # Move the stars to the frame of reference of the center of mass
# particles.pos -= particles.com_pos()
# particles.vel -= particles.com_vel()

# # Move the stars to the initial position
# particles.pos[:, 0] += initial_position * np.ones(len(particles))  # Move center of mass to x=initial_position/2, y=z=0

# # Calculate the velocity of the cluster based on the eccentricity of the orbit
# two_body = ic.ic_two_body(M_G, 1, initial_position, e=eccentricity)
# vel_cluster = two_body.vel[1,1]

# # Update vel along y direction based on eccentricity
# particles.vel[:, 1] += vel_cluster

# # calculate the mean distance between the stars to use as the softening length
# mean_distance = np.zeros((N, N)) 
# for i in range(2):
#     mean_distance += (particles.pos[:, i] - particles.pos[:, i].reshape(N, 1))**2
# mean_distance = (np.sqrt(mean_distance[mean_distance != 0.0])).mean()

# # Create the PointMass potential
# potential_PointMass = fnp.Point_Mass(Mass=M_G, softening = mean_distance) 

# # Additional potentials
# # potential_Plummer = fnp.Plummer(Mass=1, a=0.59) 

# # potential_tot = fnp.MultiPotential([potential_PointMass, potential_Plummer])

# # total time of integration
# orbital_period = 2*np.pi*np.sqrt(initial_position**3/M_G) 
# time = np.arange(0, N_orbit*orbital_period, tstep)

# # Initial total energy
# E_tot0, _, _ = particles.Etot()

# # list for analysis
# pos_list, pos_com_list, vel_list, vel_com_list, energy_list = [particles.pos], [particles.com_pos()], [particles.vel], [particles.com_vel()], [E_tot0]

# # integrate the system
# for dt in tqdm(time):

#     acc_PointMass = potential_PointMass.acceleration
#     particles, _, _, _, _ = fni.integrator_leapfrog(particles=particles, tstep=tstep, 
#                                                     acceleration_estimator=fnd.acceleration_pyfalcon, 
#                                                     external_accelerations=[potential_PointMass.acceleration])
    
#     #saving important quantities for analysis of the dynamics
#     pos_list.append(particles.pos.copy())
#     pos_com_list.append(particles.com_pos())
#     vel_list.append(particles.vel.copy())
#     vel_com_list.append(particles.com_vel())
    
#     Etot, _, _ = particles.Etot()
#     energy_list.append(Etot)

# # Convert to numpy arrays  
# pos_array = np.array(pos_list)
# pos_com_array = np.array(pos_com_list)
# vel_array = np.array(vel_list)
# vel_com_array = np.array(vel_com_list)
# energy_array = np.array(energy_list)

# # Save arrays to npy file
# array_to_save = np.zeros((len(pos_array), N, 8))

# # pos, vel, mass, energy
# array_to_save[:, :, :3], array_to_save[:, :, 3:6], array_to_save[:, :, 6], array_to_save[:, :, 7] = pos_array, vel_array, (1/N)*np.ones_like(array_to_save[:, :, 6]), energy_array

# np.save(file=f'orbit/Galactic_FoR/{N}_InitialPos_{initial_position}_e_{eccentricity}_t_{tstep}', arr=array_to_save)

# # Shift in the frame of reference of CoM and save
# position_comFoR = np.ones_like(pos_array)
# velocity_comFoR = np.ones_like(vel_array)

# for i in range(len(pos_com_array)): #for every Snapshot
#     for j in range(3):              #for x, y, z
#         position_comFoR[i][:, j] = pos_array[i][:, j] - pos_com_array[i, j] #probably it can be done with numpy array brodcast           
#         velocity_comFoR[i][:, j] = vel_array[i][:, j] - vel_com_array[i, j] #probably it can be done with numpy array brodcast 

# # with broadcast  
# # position_comFoR = pos_array - pos_com_array[:, None]
# # velocity_comFoR = vel_array - vel_com_array[:, None]    

# #save the com Frame of reference
# array_to_save = np.zeros((len(position_comFoR), N, 8))

# # pos, vel, mass, energy
# array_to_save[:, :, :3], array_to_save[:, :, 3:6], array_to_save[:, :, 6], array_to_save[:, :, 7] = position_comFoR, velocity_comFoR, (1/N)*np.ones_like(array_to_save[:, :, 6]), energy_array

# np.save(file=f'orbit/Cluster_FoR/{N}_InitialPos_{initial_position}_e_{eccentricity}_t_{tstep}', arr=array_to_save)

In [ ]:
def softening(N: int, initial_position: float, eccentricity: float) -> float:

    """
    This function calculates the softening length for the PointMass potential.  
    The softening length is calculated as the mean distance between the stars.  

    Parameters
    ----------
    N : int
        Number of stars in the cluster
    
    initial_position : float
        Initial position along the x direction
    
    eccentricity : float
        Eccentricity of the orbit

    Returns
    -------
    float
        Softening length for the PointMass potential

    """

    # Load data from CSV file
    path = './data/Plummer_Nbody_virial_' + str(N) + '.csv'
    data = pd.read_csv(path).values

    # Create Particles object
    particles = Particles(position=data[:, 4:], velocity=data[:, 1:4], mass=data[:, 0])

    # Move the stars to the frame of reference of the center of mass
    particles.pos -= particles.com_pos()
    particles.vel -= particles.com_vel()

    # Move the stars to the initial position
    particles.pos[:, 0] += initial_position * np.ones(len(particles))  # Move center of mass to x=initial_position/2, y=z=0

    # Calculate the velocity of the cluster based on the eccentricity of the orbit
    two_body = ic.ic_two_body(M_G, 1, initial_position, e=eccentricity)
    vel_cluster = two_body.vel[1,1]

    # Update vel along y direction based on eccentricity
    particles.vel[:, 1] += vel_cluster

    # calculate the mean distance between the stars to use as the softening length
    mean_distance = np.zeros((N, N)) 
    for i in range(2):
        mean_distance += (particles.pos[:, i] - particles.pos[:, i].reshape(N, 1))**2
    mean_distance = (np.sqrt(mean_distance[mean_distance != 0.0])).mean()

    return mean_distance


def orbit_integration(N: int, initial_position: float, eccentricity: float, potential: fireworks.nbodylib.potentials, tstep: float = 0.01, N_orbit: float = 2) -> None:

    """
    This function integrates the orbit of a cluster of N stars around a PointMass potential for N_orbit orbits.  
    The function moves the stars in the frame of reference of the center of mass (CoM) and then moves the stars 
    to the initial position along the x direction.  
    The function calculates the velocity of the cluster based on the eccentricity of the orbit and updates the velocity
    of the stars along the y direction.  

    The function saves the position, velocity, mass and energy of the cluster in the frame of reference of the galaxy and
    in the frame of reference of the CoM.

    Parameters
    ----------
    N : int
        Number of stars in the cluster
    
    initial_position : float
        Initial position along the x direction
    
    eccentricity : float
        Eccentricity of the orbit

    potential : fireworks.nbodylib.potentials
        Potential to be used in the integration

    tstep : float. Default = 0.01
        Time step for integration

    N_orbit : float. Default = 2
        Number of orbits to integrate

    """

    # Load data from CSV file
    path = './data/Plummer_Nbody_virial_' + str(N) + '.csv'

    data = pd.read_csv(path).values

    # Create Particles object
    particles = Particles(position=data[:, 4:], velocity=data[:, 1:4], mass=data[:, 0])

    # Move the stars to the frame of reference of the center of mass
    particles.pos -= particles.com_pos()
    particles.vel -= particles.com_vel()

    # Move the stars to the initial position
    particles.pos[:, 0] += initial_position * np.ones(len(particles))  # Move center of mass to x=initial_position/2, y=z=0

    # Calculate the velocity of the cluster based on the eccentricity of the orbit
    two_body = ic.ic_two_body(M_G, 1, initial_position, e=eccentricity)
    vel_cluster = two_body.vel[1,1]

    # Update vel along y direction based on eccentricity
    particles.vel[:, 1] += vel_cluster

    # # calculate the mean distance between the stars to use as the softening length
    # mean_distance = np.zeros((N, N)) 
    # for i in range(2):
    #     mean_distance += (particles.pos[:, i] - particles.pos[:, i].reshape(N, 1))**2
    # mean_distance = (np.sqrt(mean_distance[mean_distance != 0.0])).mean()

    # # Create the PointMass potential
    # potential_PointMass = fnp.Point_Mass(Mass=M_G, softening = mean_distance) 

    # # Additional potentials
    # # potential_Plummer = fnp.Plummer(Mass=1, a=0.59) 

    # # potential_tot = fnp.MultiPotential([potential_PointMass, potential_Plummer])

    # total time of integration
    orbital_period = 2*np.pi*np.sqrt(initial_position**3/M_G) 
    time = np.arange(0, N_orbit*orbital_period, tstep)

    # Initial total energy
    E_tot0, _, _ = particles.Etot()

    # list for analysis
    pos_list, pos_com_list, vel_list, vel_com_list, energy_list = [particles.pos], [particles.com_pos()], [particles.vel], [particles.com_vel()], [E_tot0]

    # integrate the system
    for dt in tqdm(time):

        acc_PointMass = potential.acceleration
        particles, _, _, _, _ = fni.integrator_leapfrog(particles=particles, tstep=tstep, 
                                                        acceleration_estimator=fnd.acceleration_pyfalcon, 
                                                        external_accelerations=[potential.acceleration])
        
        #saving important quantities for analysis of the dynamics
        pos_list.append(particles.pos.copy())
        pos_com_list.append(particles.com_pos())
        vel_list.append(particles.vel.copy())
        vel_com_list.append(particles.com_vel())
        
        Etot, _, _ = particles.Etot()
        energy_list.append(Etot)

    # Convert to numpy arrays  
    pos_array = np.array(pos_list)
    pos_com_array = np.array(pos_com_list)
    vel_array = np.array(vel_list)
    vel_com_array = np.array(vel_com_list)
    energy_array = np.array(energy_list)

    # Save arrays to npy file
    array_to_save = np.zeros((len(pos_array), N, 8))

    # pos, vel, mass, energy
    array_to_save[:, :, :3], array_to_save[:, :, 3:6], array_to_save[:, :, 6], array_to_save[:, :, 7] = pos_array, vel_array, (1/N)*np.ones_like(array_to_save[:, :, 6]), energy_array

    np.save(file=f'orbit/Galactic_FoR/{N}_InitialPos_{initial_position}_e_{eccentricity}_t_{tstep}', arr=array_to_save)

    # Shift in the frame of reference of CoM and save
    position_comFoR = np.ones_like(pos_array)
    velocity_comFoR = np.ones_like(vel_array)

    for i in range(len(pos_com_array)): #for every Snapshot
        for j in range(3):              #for x, y, z
            position_comFoR[i][:, j] = pos_array[i][:, j] - pos_com_array[i, j] #probably it can be done with numpy array brodcast           
            velocity_comFoR[i][:, j] = vel_array[i][:, j] - vel_com_array[i, j] #probably it can be done with numpy array brodcast 

    # with broadcast  
    # position_comFoR = pos_array - pos_com_array[:, None]
    # velocity_comFoR = vel_array - vel_com_array[:, None]    

    #save the com Frame of reference
    array_to_save = np.zeros((len(position_comFoR), N, 8))

    # pos, vel, mass, energy
    array_to_save[:, :, :3], array_to_save[:, :, 3:6], array_to_save[:, :, 6], array_to_save[:, :, 7] = position_comFoR, velocity_comFoR, (1/N)*np.ones_like(array_to_save[:, :, 6]), energy_array

    np.save(file=f'orbit/Cluster_FoR/{N}_InitialPos_{initial_position}_e_{eccentricity}_t_{tstep}', arr=array_to_save)

In [ ]:
N = 500                # Number of stars in the cluster
initial_position = 10   # Initial position along x direction 
M_G = 1e3               # Mass of the galaxy
eccentricity = 0.0    # Eccentricity of the orbit
tstep = 0.01        # Time step for integration
N_orbit = 2         # Number of orbits to integrate

mean_distance = softening(N, initial_position, eccentricity)

# Create the PointMass potential
potential_PointMass = fnp.Point_Mass(Mass=M_G, softening = mean_distance)

orbit_integration(N, initial_position, eccentricity, potential_PointMass, tstep, N_orbit)

### GIF generation for visual inspection of the integration